In [1]:
!pip install -q langchain langchain-community langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import userdata

GROQ_API_KEY = userdata.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise RuntimeError("GROQ_API_KEY not found in Colab Secrets. Please add it first.")


In [3]:
from langchain_groq import ChatGroq
from langchain_community.tools import Tool
from langchain.agents import initialize_agent

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.4,  # lower for educational guidance
    groq_api_key=GROQ_API_KEY
)


ImportError: cannot import name 'initialize_agent' from 'langchain.agents' (/usr/local/lib/python3.12/dist-packages/langchain/agents/__init__.py)

In [ ]:
def student_mentor(student_info: str) -> str:
    prompt = f"""
You are an academic mentor.

You are given a student's academic details and a predicted academic risk level.

Your tasks:
1. Explain the student's risk level in simple language.
2. Identify key academic or behavioral issues.
3. Provide exactly 3 personalized improvement steps.
4. Be supportive and motivating.

Student Details:
{student_info}
"""
    response = llm.invoke(prompt)
    return response.content


In [ ]:
tools = [
    Tool(
        name="Student Academic Mentor",
        func=student_mentor,
        description="Use this tool to provide academic guidance based on student risk data."
    )
]


In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


In [ ]:
student_query = """
Risk Level: High
Confidence: 0.82

Attendance: 60%
Study Hours: 2 hours/day
Assignments: 55%
Participation: Low
"""


In [ ]:
try:
    result = agent.invoke(student_query)

    if hasattr(result, "content"):
        print("🎓 Student Academic Guidance:\n", result.content)
    else:
        print("🎓 Student Academic Guidance:\n",
              result.get("output") or result.get("text") or result)

except Exception as e:
    print("❌ Error occurred:", e)
